In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../../')

In [ ]:
import pandas as pd
import zipfile
import os
import multiprocessing
import numpy as np
from functools import partial
import pickle

from utils.parse_pipeline import (
    _get_dir_from_filename,
    get_directories, 
    parse_profiles_for_directory,
    parse_profile,
)

from utils.parse_util import (
    HTMLFileReader,
    LinkedInProfileParser
)

In [ ]:
ARCHIVE_ROOT_DIR = '/media/sjb/Sandra M'

ARCHIVE_FILE_NAME = 'First_1000.zip'

In [ ]:
archive_file_path = os.path.join(ARCHIVE_ROOT_DIR, ARCHIVE_FILE_NAME)
archive_file_path

In [ ]:
archive_file_prefix = ARCHIVE_FILE_NAME.replace('.zip', '')
archive_file_prefix

In [ ]:
# %%time

# zf = zipfile.ZipFile(os.path.join(ARCHIVE_ROOT_DIR, 
#                                   ARCHIVE_FILE_NAME)) 

In [ ]:
PARTITION_SPLIT_MAP_DIR = ('/home/sjb/Projects/Research/'
            'LinkedIn_OB/data/partition_dir')
PARTITION_SPLIT_MAP_FILENAME = '{}|partition_map.pkl'.format(archive_file_prefix)


In [ ]:
SPLIT_NUM = 0

In [ ]:
with open(os.path.join(PARTITION_SPLIT_MAP_DIR, PARTITION_SPLIT_MAP_FILENAME), 'rb') as f:
    partition_split_map = pickle.load(f)

In [ ]:
partition_directory_list = partition_split_map[SPLIT_NUM]
len(partition_directory_list)

In [ ]:
DEST_DIR = ('/home/sjb/Projects/Research/'
            'LinkedIn_OB/data/partition_dir/temp_dir')

In [ ]:
%%time

with zipfile.ZipFile(archive_file_path, 'r')  as archive:
   # Get a list of all archived file names from the zip
   file_names = archive.namelist()
   # Iterate over the file names
   for file_name in file_names:
       # Check filename endswith csv
       if _get_dir_from_filename(file_name) in partition_directory_list:
           # Extract a single file from zip
           archive.extract(file_name, DEST_DIR)

In [ ]:
def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

In [ ]:
ts = []

count = 0

for t in os.walk(DEST_DIR):
    ts.append(t)
    
    count += 1
    if count > 3:
        break

In [ ]:
partition_archive_name = '{}_{}.zip'.format(archive_file_prefix, SPLIT_NUM)
partition_archive_file_path = os.path.join(ARCHIVE_ROOT_DIR, 'SJ_Partition', partition_archive_name)
partition_archive_file_path

In [ ]:
zipf_to_save = zipfile.ZipFile(partition_archive_file_path, 'w', zipfile.ZIP_DEFLATED)
zipdir(DEST_DIR, zipf_to_save)
zipf_to_save.close()

In [ ]:
del(archive)

In [ ]:
import gc
gc.collect()

In [ ]:
for name in dir():
    if not name.startswith('_'):
        del globals()[name]